In [1]:
%matplotlib inline

import math
import cv2

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as wd

from IPython.display import display

In [2]:
plt.rcParams["figure.figsize"] = [12, 8]

In [3]:
image_BGR = cv2.imread('/Users/yc/Downloads/IMG_1616.JPG', cv2.IMREAD_COLOR)
image_RGB_f = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2RGB).astype(np.float)
image_HSV_f = cv2.cvtColor(image_BGR, cv2.COLOR_BGR2HSV).astype(np.float)

def findMask_one_ps(image, sigma_color, color):
    '''
    依照 h s v 計算距離 輸出 0~1 代表與color相似度
    '''
    
    dis = np.abs(image - color)
    nor = dis * np.array([1.0 / 180, 1.0 / 255, 1.0 / 255])
    
    weights = np.array([20.0, 5.0, 1.0])
    nor = np.sum(nor * weights, axis = 2) / np.sum(weights)
    return 1 - nor

def findMask_one_cv(image, sigma_color, color):
    '''
    依照 h s v 計算距離 輸出 0~1 代表與color相似度
    '''
    return cv2.inRange(image, color - sigma_color, color + sigma_color).astype(np.float) / 255.0

def findMask(image, sigma_color, colors):
    '''
    params
    ======
    - image: image bitmap
    - sigma_color: color space threshold
    - colors: reference colors
    
    returns
    =======
    - mask: mask of selected pixels in [0, 1]
    '''
    mask_all = None
    for color in colors:
        
        #mask = cv2.inRange(image, color - sigma_color, color + sigma_color)
        mask = findMask_one_ps(image, sigma_color, color)
        
        if mask_all is None:
            mask_all = mask
        else:
            mask_all = cv2.max(mask, mask_all)
        
    return mask_all

In [7]:
def pickColor(h, s, v):
    select_color = np.array([h, s, v], dtype=np.float)
    display_cube = cv2.cvtColor(select_color.reshape([1, 1, 3]).astype(np.uint8), cv2.COLOR_HSV2RGB)
    
    #colors = [select_color, 
              #np.array([5, 59, 255]),
              #np.array([3, 99, 207]),
              #np.array([3, 106, 94])]
                
    colors = [select_color]
    
    ret_image = image_HSV_f.copy()
    mask = findMask(image_HSV_f, select_color, colors)
    mask *= 0.3
    mask = cv2.blur(mask, (7, 7))
    
    target = select_color
    #target = np.array([3, 99, 207], dtype=np.float)
    
    #ret_image[:,:,2][mask > 0] = ((ret_image[:,:,2][mask > 0].astype(np.float) + 207) / 2).astype(np.uint8)
    #ret_image[mask > 0] = ((ret_image[mask > 0].astype(np.float) + np.array([3, 99, 207])) / 2 ).astype(np.uint8)
    #ret_image[mask > 0] = target
    
    ret_image[:,:,1] = mask * target[1] + (1 - mask) * ret_image[:,:,1]
    ret_image[:,:,2] = mask * target[2] + (1 - mask) * ret_image[:,:,2]
    
    #plt.hist(mask.flatten(), bins=300)
    #plt.show()
    
    ret_image = cv2.cvtColor(ret_image.astype(np.uint8), cv2.COLOR_HSV2RGB)
    
    number = 4
    plt.subplot(1, number, 1); plt.imshow(display_cube)
    plt.subplot(1, number, 2); plt.imshow((mask * 255).astype(np.uint8), cmap='gray', interpolation=None)
    plt.subplot(1, number, 3); plt.imshow(ret_image)
    plt.subplot(1, number, 4); plt.imshow(image_RGB_f.astype(np.uint8))
    plt.show()

h_slide_bar = wd.IntSlider(min=0, max=180, step=1, value=10)
s_slide_bar = wd.IntSlider(min=0, max=255, step=1, value=100)
v_slide_bar = wd.IntSlider(min=0, max=255, step=1, value=149)
_ = wd.interact(pickColor, h=h_slide_bar, s=s_slide_bar, v=v_slide_bar)

In [5]:
# finite diff x
# 控制項確定

# mask 演算法
# 臉部座位點 取法
# 只調整v ?